<a href="https://colab.research.google.com/github/BanafshehHassani/ML-Handwritten-Digit-Recognition/blob/main/mnist_cnn_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook builds a classic computer vision application for identifying handwritten digits. It's trained using a simple Convolutional Neural Network on the MNIST dataset.

Data: MNIST

Author: [Banafsheh Hassani](https://https://www.linkedin.com/in/banafsheh-hassani-7b063a129/)

More Projects: [**Github**](https://github.com/BanafshehHassani/)

PyTorch on a single node
This notebook utilizes PyTorch on the Spark driver node to train the neural network on the MNIST handwritten digit recognition data.

It's adapted from the PyTorch project under the license with slight modifications in the comments. Thanks to the developers of PyTorch for this example.



In [3]:
# Import libraries

from __future__ import absolute_import, division, print_function, unicode_literals
from collections import namedtuple

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Use GPU if available

USE_GPU = torch.cuda.is_available()

MNIST_DIR = '/tmp/data/mnist'
use_cuda = USE_GPU and torch.cuda.is_available()

Params = namedtuple('Params', ['batch_size', 'test_batch_size', 'epochs', 'lr', 'momentum', 'seed', 'cuda', 'log_interval'])
args = Params(batch_size=64, test_batch_size=1000, epochs=10, lr=0.01, momentum=0.5, seed=1, cuda=use_cuda, log_interval=200)

# Data processing-MNIST
# - Download
# - Shuffle rows
# - Create batches
# - Standardize the features

torch.manual_seed(args.seed)

data_transform_fn = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))])

train_loader = torch.utils.data.DataLoader(
                 datasets.MNIST(MNIST_DIR, train=True, download=True,
                   transform=data_transform_fn),
               batch_size=args.batch_size, shuffle=True, num_workers=1)

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(MNIST_DIR, train=False,
                       transform=data_transform_fn),
        batch_size=args.test_batch_size, shuffle=True, num_workers=1)

# Build a CNN model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = Net()
model.share_memory()

# Training the model

def train_epoch(epoch, args, model, data_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(data_loader):
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(data_loader.dataset),
                100. * batch_idx / len(data_loader), loss.data.item()))


def test_epoch(model, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in data_loader:
            if args.cuda:
                data, target = data.cuda(), target.cuda()
            data, target = Variable(data), Variable(target)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').data.item() # sum up batch loss
            pred = output.data.max(1)[1] # get the index of the max log-probability
            correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(data_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))


# Run training loop over epochs, evaluate after each.

if args.cuda:
    model = model.cuda()
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
for epoch in range(1, args.epochs + 1):
    train_epoch(epoch, args, model, train_loader, optimizer)
    test_epoch(model, test_loader)


100%|██████████| 9912422/9912422 [00:00<00:00, 76094425.47it/s]


Extracting /tmp/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /tmp/data/mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 19375510.85it/s]


Extracting /tmp/data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /tmp/data/mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 23681639.92it/s]


Extracting /tmp/data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /tmp/data/mnist/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 6779547.60it/s]


Extracting /tmp/data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /tmp/data/mnist/MNIST/raw

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.371851
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.208917
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.603620
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.730665
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.473970

Test set: Average loss: 0.2090, Accuracy: 9401/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.372890
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.382565
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.394691
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.311141
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.346244

Test set: Average loss: 0.1251, Accuracy: 9619/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.443347
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.302536
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.300118
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.448292
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.358607

Test set: Average loss: 0.1005, Accuracy: 9687/10

The key information from training output:

In the prescribed logging intervals, the training process prints out how many samples it has gone through in the current epoch, as well as the loss of the current training mini-batch. It's